<a href="https://colab.research.google.com/github/chloebh9/Curse-detection-data/blob/master/Curse_detection_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd "/content/drive/My Drive/Curse-detection-data"

/content/drive/My Drive/Curse-detection-data


In [5]:
!git clone https://github.com/chloebh9/Curse-detection-data.git

fatal: destination path 'Curse-detection-data' already exists and is not an empty directory.


In [6]:
!pip install transformers torch datasets scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

데이터 전처리

In [7]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

file_path = '/content/drive/MyDrive/Curse-detection-data/Curse-detection-data/dataset.txt'

with open(file_path, 'r', encoding='utf-8') as f:
    data = f.readlines()

# |기준으로 앞의 요소는 text에, 나머지는 label에 저장
data = [line.strip().rsplit('|', maxsplit=1) for line in data if '|' in line]
df = pd.DataFrame(data, columns=['text', 'label'])
df['label'] = df['label'].astype(int)

# 전처리 함수
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # 다중 공백 제거
    text = re.sub(r'[^\w\s]', '', text)  # 특수문자 제거
    return text

df['text'] = df['text'].apply(preprocess_text)

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2)

BERT 토크나이저와 입력 준비

In [8]:
from transformers import BertTokenizer
import torch

# model_name = 'bert-base-multilingual-cased'
model_name = 'beomi/kcbert-base'

tokenizer = BertTokenizer.from_pretrained(model_name)

# 데이터셋을 토큰화된 형태로 변환
def tokenize_data(texts, labels, max_length=128):
    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )
    inputs['labels'] = torch.tensor(labels)
    return inputs

train_data = tokenize_data(X_train.tolist(), y_train.tolist())
test_data = tokenize_data(X_test.tolist(), y_test.tolist())

In [ ]:
print(train_data[:1])

{'input_ids': tensor([[  101,  9755,  9609, 46216,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

데이터 로더 생성

In [9]:
from torch.utils.data import DataLoader, TensorDataset

# input_ids: 토큰화된 입력 시퀀스
# attention_mask : 패딩 토큰 구분
# labels: 해당 시퀀스의 욕설 여부
train_dataset = TensorDataset(train_data['input_ids'], train_data['attention_mask'], train_data['labels'])
test_dataset = TensorDataset(test_data['input_ids'], test_data['attention_mask'], test_data['labels'])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
print(train_dataset[:1])

(tensor([[  101,  9755,  9609, 46216,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0, 

BERT 모델 학습

In [10]:
import torch
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

# model_name = 'bert-base-multilingual-cased'   # 다국어
model_name = 'beomi/kcbert-base'                # 한국어만
# BERT 모델 로드
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

epoch = 5
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * epoch
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

model.train()
for epoch in range(epoch):
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
    print(f'Epoch {epoch + 1} completed')

model.safetensors:   5%|4         | 21.0M/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1 completed
Epoch 2 completed
Epoch 3 completed
Epoch 4 completed
Epoch 5 completed


모델 평가

In [11]:
from sklearn.metrics import accuracy_score, classification_report

# 평가 모드
model.eval()

predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
report = classification_report(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')

Accuracy: 0.8952789699570816
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.92      0.92       751
           1       0.85      0.85      0.85       414

    accuracy                           0.90      1165
   macro avg       0.89      0.89      0.89      1165
weighted avg       0.90      0.90      0.90      1165



코드 합친거

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, classification_report

# 데이터 로드 및 전처리
file_path = '/content/drive/MyDrive/Curse-detection-data/Curse-detection-data/dataset.txt'

with open(file_path, 'r', encoding='utf-8') as f:
    data = f.readlines()

data = [line.strip().rsplit('|', maxsplit=1) for line in data if '|' in line]
df = pd.DataFrame(data, columns=['text', 'label'])
df['label'] = df['label'].astype(int)

def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

df['text'] = df['text'].apply(preprocess_text)

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2)

model_name = 'beomi/kcbert-base'
tokenizer = BertTokenizer.from_pretrained(model_name)

def tokenize_data(texts, labels, max_length=128):
    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )
    inputs['labels'] = torch.tensor(labels)
    return inputs

train_data = tokenize_data(X_train.tolist(), y_train.tolist())
test_data = tokenize_data(X_test.tolist(), y_test.tolist())

train_dataset = TensorDataset(train_data['input_ids'], train_data['attention_mask'], train_data['labels'])
test_dataset = TensorDataset(test_data['input_ids'], test_data['attention_mask'], test_data['labels'])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

epoch = 5
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_loader) * epoch
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

model.train()
for epoch in range(epoch):
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
    print(f'Epoch {epoch + 1} completed')

model.eval()
predictions, true_labels = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
report = classification_report(true_labels, predictions)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{report}')